# asyncioのテスト

In [7]:
import asyncio
import time

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    task1 = asyncio.create_task(
        say_after(1, 'hello'))

    task2 = asyncio.create_task(
        say_after(2, 'world'))

    print(f"started at {time.strftime('%X')}")

    # Wait until both tasks are completed (should take
    # around 2 seconds.)
    await task1
    await task2

    print(f"finished at {time.strftime('%X')}")

await main()


started at 11:36:03
hello
world
finished at 11:36:05


### coroutineの場合

In [7]:
import asyncio


async def nested():
    return 42


async def main():
    # Nothing happens if we just call "nested()".
    # A coroutine object is created but not awaited,
    # so it *won't run at all*.
    
    # nested()

    # Let's do it differently now and await it:
    print(await nested())  # will print "42".

await main()


42


### taskの場合

In [6]:
import asyncio


async def nested():
    return 42


async def main():
    # Schedule nested() to run soon concurrently
    # with "main()".
    task = asyncio.create_task(nested())

    # "task" can now be used to cancel "nested()", or
    # can simply be awaited to wait until it is complete:
    print(await task)

await main()


42


### 複数Taskの実行

In [1]:
import asyncio


async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({number}), currently i={i}, f={f}...")
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")
    return f


async def main():
    # Schedule three calls *concurrently*:
    L = await asyncio.gather(
        factorial("A", 2),
        factorial("B", 3),
        factorial("C", 4),
    )
    print(L)

await main()


Task A: Compute factorial(2), currently i=2, f=1...
Task B: Compute factorial(3), currently i=2, f=1...
Task C: Compute factorial(4), currently i=2, f=1...
Task A: factorial(2) = 2
Task B: Compute factorial(3), currently i=3, f=2...
Task C: Compute factorial(4), currently i=3, f=2...
Task B: factorial(3) = 6
Task C: Compute factorial(4), currently i=4, f=6...
Task C: factorial(4) = 24
[2, 6, 24]


In [8]:
import asyncio
import time


async def task1(name, number):
    f = 1
    for i in range(2, number + 1):
        current_time = time.time()
        print(
            f"{current_time} Task1 {name}: Compute factorial({number}), currently i={i}, f={f}...")
        await asyncio.sleep(1)
        f *= i
    current_time = time.time()
    print(f"{current_time} Task1 {name}: factorial({number}) = {f}")
    return f


async def task2(name, number):
    f = 1
    for i in range(number):
        time.sleep(f)
        current_time = time.time()

        print(f"{current_time} Task2 {name}: factorial({number}) = {f}")
        f += 1
    return f

async def main():
    # Schedule three calls *concurrently*:

    L = await asyncio.gather(
        task1("A", 3),
        task2("B", 3),
    )
    
    current_time = time.time()

    print(f'{current_time} {L}:{type(L)}')

await main()


1686721098.8191576 Task1 A: Compute factorial(3), currently i=2, f=1...
1686721099.8199356 Task2 B: factorial(3) = 1
1686721101.8206809 Task2 B: factorial(3) = 2
1686721104.821054 Task2 B: factorial(3) = 3
1686721104.821054 Task1 A: Compute factorial(3), currently i=3, f=2...
1686721105.824202 Task1 A: factorial(3) = 6
1686721105.824202 [6, 4]:<class 'list'>


In [10]:
import asyncio
import time


async def task1(name, number):
    f = 1
    for i in range(2, number + 1):
        current_time = time.time()
        print(
            f"{current_time} Task1 {name}: Compute factorial({number}), currently i={i}, f={f}...")
        await asyncio.sleep(1)
        f *= i
    current_time = time.time()
    print(f"{current_time} Task1 {name}: factorial({number}) = {f}")
    return f


async def task2(name, number):
    f = 1
    for i in range(number):
        await asyncio.sleep(f)
        current_time = time.time()

        print(f"{current_time} Task2 {name}: factorial({number}) = {f}")
        f += 1
    return f

async def main():
    # Schedule three calls *concurrently*:

    L = await asyncio.gather(
        task1("A", 3),
        task2("B", 3),
    )
    
    current_time = time.time()

    print(f'{current_time} {L}:{type(L)}')

await main()


1686721425.874763 Task1 A: Compute factorial(3), currently i=2, f=1...
1686721426.8882246 Task1 A: Compute factorial(3), currently i=3, f=2...
1686721426.8882246 Task2 B: factorial(3) = 1
1686721427.9027603 Task1 A: factorial(3) = 6
1686721428.897211 Task2 B: factorial(3) = 2
1686721431.903201 Task2 B: factorial(3) = 3
1686721431.904241 [6, 4]:<class 'list'>


### taskの入れ子

In [11]:
import asyncio
import time


async def task1(name, number):
    f = 1
    for i in range(2, number + 1):
        current_time = time.time()
        print(
            f"{current_time} Task1 {name}: Compute factorial({number}), currently i={i}, f={f}...")
        await asyncio.sleep(1)
        task_2 = asyncio.create_task(task2('B', 3))
        await task_2
        f *= i
    current_time = time.time()
    print(f"{current_time} Task1 {name}: factorial({number}) = {f}")
    return f


async def task2(name, number):
    f = 1
    for i in range(number):
        await asyncio.sleep(f)
        current_time = time.time()

        print(f"{current_time} Task2 {name}: factorial({number}) = {f}")
        f += 1
    return f

async def main():
    # Schedule three calls *concurrently*:
    task_1 = asyncio.create_task(task1('A', 3))

    r = await task_1

    current_time = time.time()

    print(f'{current_time} {r}:{type(r)}')

await main()


1686721524.4652464 Task1 A: Compute factorial(3), currently i=2, f=1...
1686721526.4812405 Task2 B: factorial(3) = 1
1686721528.4917595 Task2 B: factorial(3) = 2
1686721531.5050256 Task2 B: factorial(3) = 3
1686721531.5050256 Task1 A: Compute factorial(3), currently i=3, f=2...
1686721533.5204754 Task2 B: factorial(3) = 1
1686721535.5314631 Task2 B: factorial(3) = 2
1686721538.5409384 Task2 B: factorial(3) = 3
1686721538.5409384 Task1 A: factorial(3) = 6
1686721538.5409384 6:<class 'int'>
